# HOMEWORK 3

In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings

from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,6)
plt.rcParams['font.size'] = 15
plt.rcParams['legend.fontsize'] = 13

import sys
sys.path.insert(0, '../cmds')
from treasury_cmds import *

In [2]:
USE_PAR_TBOND = False
# Set coupon on 30-year T bond to equal Nov 4 YTM
# Or stick with the coupon rate in the case, for the Aug 2008 T bond

YTM = [0.04193, .0436]

if USE_PAR_TBOND:
    CPNRATE = [YTM[0], .0436]
else:
    CPNRATE = [0.0450, .0436]

SWAPRATE = [.042560, .0408]

TPRICE = 105
PAR = 100

NOTIONAL = 500e6
HAIRCUT = .02

DTIME = .5
tau0 = 30
tau1 = tau0-DTIME

1.1)

In [3]:
summary = pd.DataFrame(index=['coupon rate','YTM','swap rate','spread'],columns = ['Nov 2008','May 2009'],dtype=float)
summary.loc['coupon rate'] = CPNRATE
summary.loc['YTM'] = YTM
summary.loc['swap rate'] = SWAPRATE
summary.loc['YTM spread'] = summary.loc['swap rate'] - summary.loc['YTM']
summary.loc['coupon spread'] = summary.loc['swap rate'] - summary.loc['coupon rate']
summary.style.format('{:.2%}')

,Nov 2008,May 2009
coupon rate,4.50%,4.36%
YTM,4.19%,4.36%
swap rate,4.26%,4.08%
spread,nan%,nan%
YTM spread,0.06%,-0.28%
coupon spread,-0.24%,-0.28%


In [4]:
SOFR = np.nan
CF = pd.DataFrame(index=['T bond','Repo','Swap (floating leg)','Swap (fixed leg)'],columns=['May 2009'],dtype=float)
CF.loc['Repo'] -SOFR
CF.loc['Swap (floating leg)'] = SOFR
CF.loc[['T bond']] = PAR * CPNRATE[0] /2
CF.loc[['Swap (fixed leg)']] = -PAR * SWAPRATE[0]/2
CF.loc['Net Payment'] = CF.sum(axis=0)
CF.style.format('${:,.2f}')

,May 2009
T bond,$2.25
Repo,$nan
Swap (floating leg),$nan
Swap (fixed leg),$-2.13
Net Payment,$0.12


In [5]:
def duration_closed_formula(tau, ytm, cpnrate=None, freq=2):

    if cpnrate is None:
        cpnrate = ytm
        
    y = ytm/freq
    c = cpnrate/freq
    T = tau * freq
        
    if cpnrate==ytm:
        duration = (1+y)/y  * (1 - 1/(1+y)**T)
        
    else:
        duration = (1+y)/y - (1+y+T*(c-y)) / (c*((1+y)**T-1)+y)

    duration /= freq
    
    return duration

In [6]:
tab_duration = pd.DataFrame(dtype=float, index=['T bond','fixed leg', 'floating leg'], columns=['duration'])
tab_duration.loc['T bond'] = duration_closed_formula(30, summary.loc['YTM','Nov 2008'], summary.loc['coupon rate','Nov 2008'])
tab_duration.loc['fixed leg'] = duration_closed_formula(30, summary.loc['swap rate','Nov 2008'])
tab_duration.loc['floating leg'] = .5

tab_duration['dollar duration'] = tab_duration['duration'] * np.array([TPRICE, PAR, PAR])

tab_duration.loc['swap'] = tab_duration.loc['fixed leg'] - tab_duration.loc['floating leg']

tab_duration.loc['net'] = tab_duration.loc['T bond'] - tab_duration.loc['swap']

tab_duration

,duration,dollar duration
T bond,17.083633,1793.781472
fixed leg,17.212744,1721.274445
floating leg,0.500000,50.000000
swap,16.712744,1671.274445
net,0.370889,122.507027


1.3)

In [7]:
hedge_ratio = tab_duration.loc['swap','dollar duration'] / tab_duration.loc['T bond','dollar duration']
contracts = pd.DataFrame(NOTIONAL * np.array([hedge_ratio /TPRICE, -1/PAR]), index=['T bond','swap'], columns=['positions'])
contracts

,positions
T bond,4.436689e+06
swap,-5.000000e+06


1.4)

The rising YTM on the T bond suggests...

lower (adjusted) price for the T-bond.
we are long the T-bond, so this would be a loss.
The lower swap rate suggests...

higher value on the fixed leg of a swap.
we are paying fixed, (ie short the fixed leg), so this would again be a loss.
The floating leg of the swap and repo rate cancel

1.5)

In [8]:
def price_treasury_ytm(time_to_maturity, ytm, cpn_rate,freq=2,face=100):
    c = cpn_rate/freq
    y = ytm/freq
    
    tau = round(time_to_maturity * freq)
    
    pv = 0
    for i in range(1,tau):
        pv += 1 / (1+y)**i
    
    pv = c*pv + (1+c)/(1+y)**tau
    pv *= face
    
    return pv

In [9]:
prices = pd.DataFrame(index=['T bond', 'swap'],dtype=float,columns=['Nov 2008'])

if USE_PAR_TBOND:
    prices.loc['T bond','Nov 2008'] = price_treasury_ytm(tau0, summary.loc['YTM','Nov 2008'], summary.loc['coupon rate','Nov 2008'])
else:
    prices.loc['T bond','Nov 2008'] = TPRICE

prices.loc['swap','Nov 2008'] = PAR - PAR

prices.loc['T bond','May 2009'] = price_treasury_ytm(tau1, summary.loc['YTM','May 2009'], summary.loc['coupon rate','Nov 2008'])
prices.loc['swap','May 2009'] = price_treasury_ytm(tau1, summary.loc['swap rate','May 2009'], summary.loc['swap rate','Nov 2008']) - PAR

prices.style.format('${:,.2f}')

,Nov 2008,May 2009
T bond,$105.00,$102.31
swap,$0.00,$3.00


1.6)

In [11]:
pnl=pd.DataFrame(dtype=float,index=['T bond','swap'],columns=['cashflow'])

pnl['cashflow'] = CF.loc[['T bond','Swap (fixed leg)']].values * contracts.abs().values
pnl['capital gains'] = prices.diff(axis=1)['May 2009'].values * contracts.values[:,0]

pnl.loc['net'] = pnl.sum()

pnl['total'] = pnl.sum(axis=1)

pnl.style.format('${:,.2f}',na_rep='')

,cashflow,capital gains,total
T bond,"$9,982,549.26","$-11,928,477.82","$-1,945,928.55"
swap,"$-10,640,000.00","$-15,016,747.03","$-25,656,747.03"
net,"$-657,450.74","$-26,945,224.84","$-27,602,675.58"


In [12]:
capital = pd.DataFrame(prices.iloc[:,0].values * contracts.values[:,0], index=['T bond','swap'],columns=['assets'])
capital['equity'] = capital['assets'] * HAIRCUT
capital.loc['net'] = capital.sum()

capital['pnl'] = pnl['total']
capital['return'] = capital['pnl']/capital['equity']
capital.loc[['T bond','swap'],'return'] = np.nan

capital.style.format({'assets':'${:,.2f}','equity':'${:,.2f}','pnl':'${:,.2f}','return':'{:.2%}'},na_rep='')

,assets,equity,pnl,return
T bond,"$465,852,298.97","$9,317,045.98","$-1,945,928.55",
swap,$-0.00,$-0.00,"$-25,656,747.03",
net,"$465,852,298.97","$9,317,045.98","$-27,602,675.58",-296.26%
